<a href="https://colab.research.google.com/github/Usually-zz/2021_IMC/blob/main/dgmm_em.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import scipy.stats as stats ## for mvn
from sklearn.cluster import KMeans
import seaborn as sns

In [2]:
iris = sns.load_dataset("iris")
X = iris.iloc[:,:4]
X = np.asarray(X)
y = iris.iloc[:,4]

n, p = X.shape
c = 3

In [3]:
kmeans = KMeans(n_clusters=c, random_state=0).fit(X)

In [4]:
mu_c = torch.tensor(kmeans.cluster_centers_)

In [5]:
mu_c

tensor([[6.8500, 3.0737, 5.7421, 2.0711],
        [5.0060, 3.4280, 1.4620, 0.2460],
        [5.9016, 2.7484, 4.3935, 1.4339]], dtype=torch.float64)

In [5]:
n_c = torch.zeros(3)
co_c = torch.zeros(c, 4, 4)

for j in range(c):
  indx = kmeans.labels_ == j
  d = X[indx]
  n_c[j] = sum(indx)
  co_c[j] = torch.from_numpy(np.cov(d.T))

In [7]:
n_c

tensor([38., 50., 62.])

In [6]:
pi_c = n_c/sum(n_c)
pi_c

tensor([0.2533, 0.3333, 0.4133])

In [7]:
co_c

tensor([[[0.2442, 0.0254, 0.1773, 0.0055],
         [0.0254, 0.0842, 0.0122, 0.0344],
         [0.1773, 0.0122, 0.2387, 0.0253],
         [0.0055, 0.0344, 0.0253, 0.0783]],

        [[0.1242, 0.0992, 0.0164, 0.0103],
         [0.0992, 0.1437, 0.0117, 0.0093],
         [0.0164, 0.0117, 0.0302, 0.0061],
         [0.0103, 0.0093, 0.0061, 0.0111]],

        [[0.2175, 0.0637, 0.1343, 0.0345],
         [0.0637, 0.0878, 0.0549, 0.0290],
         [0.1343, 0.0549, 0.2590, 0.1214],
         [0.0345, 0.0290, 0.1214, 0.0885]]])

In [7]:
dd = (torch.from_numpy(np.array(X[indx])).float() - mu_c[j]).T
torch.matmul(dd, dd.T) / sum(indx)

tensor([[0.2140, 0.0627, 0.1321, 0.0340],
        [0.0627, 0.0864, 0.0540, 0.0285],
        [0.1321, 0.0540, 0.2548, 0.1194],
        [0.0340, 0.0285, 0.1194, 0.0871]], dtype=torch.float64)

In [8]:
n_cc = torch.tensor([[10, 20, 30], [0, 15, 40], [0, 0, 35]])

In [9]:
pi_cc = n_cc/torch.sum(n_cc)

In [10]:
pi_cc

tensor([[0.0667, 0.1333, 0.2000],
        [0.0000, 0.1000, 0.2667],
        [0.0000, 0.0000, 0.2333]])

In [10]:
#initialize
pi_d = .4
#pi_c[j]
#pi_cc[j][k]

In [12]:
### E-step:
## calculate 1) pi_i => d; 2) pi_j => z; 3) pi_jk => gamma
## first compute 2) and 3) and then 1)

r_ij_top = torch.zeros(c, n)
r_ijk_top = torch.zeros(c, c, n)
    
for j in range(c):
  r_ij_top[j] = pi_d * (pi_c[j] * stats.multivariate_normal(mu_c[j], co_c[j]).pdf(X))
  for k in range(c):
    r_ijk_top[j,k] = (1-pi_d) * (pi_cc[j][k] * stats.multivariate_normal(mu_c[j] + mu_c[k], co_c[j] + co_c[k]).pdf(X))

r_ij_bot = torch.sum(r_ij_top, 0) #for 2)
r_ij = r_ij_top / r_ij_bot

r_ijk_bot = torch.sum(torch.sum(r_ijk_top, 0), 0) #for 3)
r_ijk = r_ijk_top / r_ijk_bot

r_i = r_ij_bot / (r_ij_bot + r_ijk_bot) #for 1

## compute log-likelihood
logl = torch.sum(torch.log(r_ij_bot) + torch.log(r_ijk_bot))

## predictions
#d0pred = torch.max(r_ij, 0).indices
#d0pred

#d1pred = torch.zeros(n, 2)
#for i in range(n):
#  d1pred[i,:] = torch.tensor(np.unravel_index(r_ijk[:,:,i].argmax(), r_ijk[:,:,i].shape))
#d1pred

### M-step:

## update pi_d
pi_d = torch.sum(r_i) / n #(one value)

## update pi_c (c values)
n_c = torch.sum(r_i * r_ij, 1) ## number of points in n_c
pi_c = n_c / sum(n_c)
#pi_c = r_i * r_ij / n

## update pi_cc (cxc upper triangular matrix)
n_cc = torch.sum(((1 - r_i) * r_ijk).T, 0).T ## number of points in each n_cc
pi_cc = n_cc / torch.sum(n_cc)
#pi_cc = (1 - r_i) * r_ijk / n

## update mu_cj and sigma_cj
bot1 = 0; bot2 = 0
m_top1 = 0; c_top1 = 0
m_top2 = 0; c_top2 = 0

## note i and j position switch for r_ijk
for j in range(c):

  for i in range(n):
    
    w1 = r_i[i] * r_ij[j,i]
    bot1 += w1
    
    m_top1 += w1 * torch.tensor([X[i,:]])

    top_ker1 = torch.tensor([X[i,:]]) - mu_c[j,:]
    c_top1 += w1 * (top_ker1.T * top_ker1)

    for k in range(c):
      w2 = (1-r_i[i]) * r_ijk[j,k,i]
      bot2 += w2
      
      m_top2 += w2 * (torch.tensor([X[i,:]]) - mu_c[k,:])
      
      top_ker2 = torch.tensor([X[i,:]]) - (mu_c[j,:] + mu_c[k,:])    
      c_top2 += w2 * ((top_ker2.T * top_ker2) - co_c[k,:])

  mu_c[j] = (m_top1 + m_top2) / (bot1 + bot2)
  co_c[j] = (c_top1 + c_top2) / (bot1 + bot2)

In [13]:
mu_c

tensor([[6.7986, 3.0524, 5.7335, 2.0876],
        [5.7689, 3.2681, 3.2798, 1.0297],
        [5.8433, 3.0573, 3.7580, 1.1993]], dtype=torch.float64)

In [14]:
co_c

tensor([[[0.3064, 0.0464, 0.2257, 0.0087],
         [0.0464, 0.0929, 0.0299, 0.0367],
         [0.2257, 0.0299, 0.2502, 0.0147],
         [0.0087, 0.0367, 0.0147, 0.0751]],

        [[0.2003, 0.0756, 0.1052, 0.0095],
         [0.0756, 0.1204, 0.0193, 0.0208],
         [0.1052, 0.0193, 0.1234, 0.0097],
         [0.0095, 0.0208, 0.0097, 0.0382]],

        [[0.2191, 0.0757, 0.1310, 0.0268],
         [0.0757, 0.1075, 0.0398, 0.0266],
         [0.1310, 0.0398, 0.1914, 0.0589],
         [0.0268, 0.0266, 0.0589, 0.0563]]])

In [ ]:
class DGMM(object):
  
  def __init__(self, X, c=3):
    self.X = np.asarray(X)
    
    # dimension
    self.n, self.p = X.shape
    self.c = c
  
  def _init(self):
    self.r_i = torch.ones(self.n)
    self.r_ij = torch.ones(self.c, self.n)
    self.r_ijk = torch.ones(self.c, self.c, self.n)

    self.n_c = torch.zeros(self.c)
    self.n_cc = torch.zeros(self.c)

    self.pi_d = torch.ones(1)
    self.pi_c = torch.ones(self.c) / self.c
    self.pi_cc = torch.triu(torch.ones(self.c, self.c)) / (self.c * self.c)

    self.mu_c = torch.ones(self.c, self.p)
    self.co_c = torch.zeros(self.c, self.p, self.p)

  def estep(self):

    ### E-step:
    ## input: n, p, c, pi_d, pi_c, pi_cc, mu_c, co_c
    ## output: r_i, r_ij, r_ijk, ll
    ## calculate 1) pi_i => d; 2) pi_j => z; 3) pi_jk => gamma
    ## first compute 2) and 3) and then 1)

    r_ij_top = torch.zeros(self.c, self.n)
    r_ijk_top = torch.zeros(self.c, self.c, self.n)
    
    for j in range(self.c):
      r_ij_top[j] = self.pi_d * (self.pi_c[j] * stats.multivariate_normal(self.mu_c[j], self.co_c[j]).pdf(self.X))
      for k in range(c):
        r_ijk_top[j,k] = (1-self.pi_d) * (self.pi_cc[j,k] * stats.multivariate_normal(self.mu_c[j] + self.mu_c[k], self.co_c[j] + self.co_c[k]).pdf(self.X))

    r_ij_bot = torch.sum(r_ij_top, 0) #for 2)
    self.r_ij = r_ij_top / r_ij_bot

    r_ijk_bot = torch.sum(torch.sum(r_ijk_top, 0), 0) #for 3)
    self.r_ijk = r_ijk_top / r_ijk_bot

    self.r_i = r_ij_bot / (r_ij_bot + r_ijk_bot) #for 1

    ## compute log-likelihood
    logl = torch.sum(torch.log(r_ij_bot) + torch.log(r_ijk_bot))

    ## predictions
    #d0pred = torch.max(r_ij, 0).indices
    #d0pred

    #d1pred = torch.zeros(n, 2)
    #for i in range(n):
    #  d1pred[i,:] = torch.tensor(np.unravel_index(r_ijk[:,:,i].argmax(), r_ijk[:,:,i].shape))
    #d1pred

    return(logl)

  def mstep(self):
    
    ### M-step:
    ## input: n, p, c, r_i, r_ij, r_ijk
    ## output: pi_d, n_c, pi_c, n_cc, pi_cc, mu_c, co_c
    ## update pi_d
    self.pi_d = torch.sum(self.r_i) / self.n #(one value)

    ## update pi_c (c values)
    self.n_c = torch.sum(self.r_i * self.r_ij, 1) ## number of points in n_c
    self.pi_c = self.n_c / sum(self.n_c)
    #pi_c = r_i * r_ij / n

    ## update pi_cc (cxc upper triangular matrix)
    self.n_cc = torch.sum(((1 - self.r_i) * self.r_ijk).T, 0).T ## number of points in each n_cc
    self.pi_cc = self.n_cc / torch.sum(self.n_cc)
    #pi_cc = (1 - r_i) * r_ijk / n

    ## update mu_cj and sigma_cj
    bot1 = 0; bot2 = 0
    m_top1 = 0; c_top1 = 0
    m_top2 = 0; c_top2 = 0

    ## note i and j position switch for r_ijk
    for j in range(self.c):
      
      for i in range(self.n):

        w1 = self.r_i[i] * self.r_ij[j,i]
        bot1 += w1

        m_top1 += w1 * torch.tensor([self.X[i,:]])

        top_ker1 = torch.tensor([self.X[i,:]]) - self.mu_c[j,:]
        c_top1 += w1 * (top_ker1.T * top_ker1)

        for k in range(self.c):
          w2 = (1-self.r_i[i]) * self.r_ijk[j,k,i]
          bot2 += w2

          m_top2 += w2 * (torch.tensor([self.X[i,:]]) - self.mu_c[k,:])

          top_ker2 = torch.tensor([self.X[i,:]]) - (self.mu_c[j,:] + self.mu_c[k,:])    
          c_top2 += w2 * ((top_ker2.T * top_ker2) - self.co_c[k,:])

      self.mu_c[j] = (m_top1 + m_top2) / (bot1 + bot2)
      self.co_c[j] = (c_top1 + c_top2) / (bot1 + bot2)


In [92]:
class DGMM(object):
  
  def __init__(self, X, c=3):
    self.X = np.asarray(X)
    
    # dimension
    self.n, self.p = X.shape
    self.c = c
  
  def _init(self):
    self.r_i = torch.ones(self.n)
    self.r_ij = torch.ones(self.c, self.n)
    self.r_ijk = torch.ones(self.c, self.c, self.n)

    self.n_c = torch.zeros(self.c)
    self.n_cc = torch.zeros(self.c)

    self.pi_d = torch.ones(1)
    self.pi_c = torch.ones(self.c) / self.c
    self.pi_cc = torch.triu(torch.ones(self.c, self.c)) / (self.c * self.c)

    self.mu_c = torch.ones(self.c, self.p)
    self.co_c = torch.zeros(self.c, self.p, self.p)
  
  def train(self, itermax=100):
    
    self._init() 
    #self.gmmKmeansInitial() ## work on this!!!
    
    llv = [0.0]
    for iter in range(itermax):
      lli = self.estep()
      self.mstep()
      
      print('Iteration', iter + 1, 'Likelihood: ', lli)
        
      if abs(llv[-1] - lli) < 1e-4:
        break 
      
      llv.append(lli)
    return llv[1:]

  def estep(self):
    
    ### E-step:
    ## calculate 1) pi_i => d; 2) pi_j => z; 3) pi_jk => gamma
    ## first compute 2) and 3) and then 1)
    
    r_ij_top = torch.zeros(self.c, self.n)
    r_ijk_top = torch.zeros(self.c, self.c, self.n)
    
    for j in range(self.c):
      r_ij_top[j] = self.pi_d * (self.pi_c[j] * stats.multivariate_normal(self.mu_c[j], self.co_c[j]).pdf(self.X))
      for k in range(self.c):
        r_ijk_top[j][k] = (1-self.pi_d) * (self.pi_cc[j][k] * stats.multivariate_normal(self.mu_c[j] + self.mu_c[k], self.co_c[j] + self.co_c[k]).pdf(self.X))

    ## for 2)
    #r_ij_top = torch.zeros(c, n)
    #for j in range(c):
    #  r_ij_top[j] = pi_d * (pi_c[j] * stats.multivariate_normal(mu_c[j], co_c[j]).pdf(X))

    r_ij_bot = torch.sum(r_ij_top, 0) #for 1)
    self.r_ij = r_ij_top / r_ij_bot
  
    ## for 3)
    #r_ijk_top = torch.zeros(c, c, n) 
    #for j in range(c):
    #  for k in range(c):
    #    r_ijk_top[j][k] = (1-pi_d) * (pi_cc[j][k] * stats.multivariate_normal(mu_c[j] + mu_c[k], co_c[j] + co_c[k]).pdf(X))

    r_ijk_bot = torch.sum(torch.sum(r_ijk_top, 0), 0) #for 1)
    self.r_ijk = r_ijk_top / r_ijk_bot

    ## for 1)
    self.r_i = r_ij_bot / (r_ij_bot + r_ijk_bot)

    ## compute log-likelihood
    logl = torch.sum(torch.log(r_ij_bot) + torch.log(r_ijk_bot))

    ## predictions
    pred0 = torch.max(self.r_ij, 0).indices
    #d0pred

    pred1 = torch.zeros(self.n, 2)
    for i in range(self.n):
      pred1[i,:] = torch.tensor(np.unravel_index(self.r_ijk[:,:,i].argmax(), self.r_ijk[:,:,i].shape))
    #d1pred

    return(logl, pred0, pred1)

  def mstep(self):

    ### M-step:
  
    ## update pi
    self.pi_d = torch.sum(self.r_i) / n #(one value)
  
    ## update pi_c (c values)
    self.n_c = torch.sum(self.r_i * self.r_ij, 1) ## number of points in n_c
    self.pi_c = self.n_c / sum(self.n_c)
    #pi_c = r_i * r_ij / n
  
    ## update pi_cc (cxc upper triangular matrix)
    self.n_cc = torch.sum(((1 - self.r_i) * self.r_ijk).T, 0).T ## number of points in each n_cc
    self.pi_cc = self.n_cc / torch.sum(self.n_cc)
    #pi_cc = (1 - r_i) * r_ijk / n
  
    ## update mu_cj and sigma_cj
    bot1 = 0; bot2 = 0
    m_top1 = 0; c_top1 = 0
    m_top2 = 0; c_top2 = 0
  
    for j in range(c):
    
      for i in range(n):
        w1 = r_i[i] * r_ij[j,i]
        bot1 += w1
    
        m_top1 += w1 * torch.tensor([X.iloc[i,:]])

        top_ker1 = torch.tensor([X.iloc[i,:]]) - mu_c[j,:]
        c_top1 += w1 * (top_ker1.T * top_ker1)

        for k in range(c):
          w2 = (1-r_i[i]) * r_ijk[j,k,i]
          bot2 += w2
      
          m_top2 += w2 * (torch.tensor([X.iloc[i,:]]) - mu_c[k,:])
      
          top_ker2 = torch.tensor([X.iloc[i,:]]) - (mu_c[j,:] + mu_c[k,:])    
          c_top2 += w2 * ((top_ker2.T * top_ker2) - co_c[k,:])

      self.mu_c[j] = (m_top1 + m_top2) / (bot1 + bot2)
      self.co_c[j] = (c_top1 + c_top2) / (bot1 + bot2)

    #return(pi_d, n_c, pi_c, n_cc, pi_cc, mu_c, co_c)

  


In [66]:
n = 150

### E-step:
## calculate 1) pi_i => d; 2) pi_j => z; 3) pi_jk => gamma
## first compute 2) and 3) and then 1)

## for 2)
r_ij_top = torch.zeros(c, n) 
for j in range(c):
  r_ij_top[j] = pi_d * (pi_c[j] * stats.multivariate_normal(mu_c[j], co_c[j]).pdf(X))

r_ij_bot = torch.sum(r_ij_top, 0) #for 1)
r_ij = r_ij_top / r_ij_bot

## for 3)
r_ijk_top = torch.zeros(c, c, n) 
for j in range(c):
  for k in range(c):
    r_ijk_top[j][k] = (1-pi_d) * (pi_cc[j][k] * 
                             stats.multivariate_normal(mu_c[j] + mu_c[k], co_c[j] + co_c[k]).pdf(X))

r_ijk_bot = torch.sum(torch.sum(r_ijk_top, 0), 0) #for 1)
r_ijk = r_ijk_top / r_ijk_bot

r_i = r_ij_bot / (r_ij_bot + r_ijk_bot)

## compute log-likelihood
logl = torch.sum(torch.log(r_ij_bot) + torch.log(r_ijk_bot))

## predictions
d0pred = torch.max(r_ij, 0).indices
#d0pred

d1pred = torch.zeros(n, 2)
for i in range(n):
  d1pred[i,:] = torch.tensor(np.unravel_index(r_ijk[:,:,i].argmax(), r_ijk[:,:,i].shape))
#d1pred

### E-step:

## update pi
pi_d = torch.sum(r_i) / n #(one value)

## update pi_c (c values)
n_c = torch.sum(r_i * r_ij, 1) ## number of points in n_c
pi_c = n_c / sum(n_c)
#pi_c = r_i * r_ij / n

## update pi_cc (cxc upper triangular matrix)
n_cc = torch.sum(((1 - r_i) * r_ijk).T, 0).T ## number of points in each n_cc
pi_cc = n_cc / torch.sum(n_cc)
#pi_cc = (1 - r_i) * r_ijk / n

## update mu_cj and sigma_cj
bot1 = 0; bot2 = 0
m_top1 = 0; c_top1 = 0
m_top2 = 0; c_top2 = 0

for j in range(c):

  for i in range(n):
    
    w1 = r_i[i] * r_ij[j,i]
    bot1 += w1
    
    m_top1 += w1 * torch.tensor([X.iloc[i,:]])

    top_ker1 = torch.tensor([X.iloc[i,:]]) - mu_c[j,:]
    c_top1 += w1 * (top_ker1.T * top_ker1)

    for k in range(c):
      w2 = (1-r_i[i]) * r_ijk[j,k,i]
      bot2 += w2
      
      m_top2 += w2 * (torch.tensor([X.iloc[i,:]]) - mu_c[k,:])
      
      top_ker2 = torch.tensor([X.iloc[i,:]]) - (mu_c[j,:] + mu_c[k,:])    
      c_top2 += w2 * ((top_ker2.T * top_ker2) - co_c[k,:])

  mu_c[j] = (m_top1 + m_top2) / (bot1 + bot2)
  co_c[j] = (c_top1 + c_top2) / (bot1 + bot2)

In [67]:
pi_d

tensor(1.)

In [68]:
pi_c

tensor([0.2469, 0.3333, 0.4197])

In [69]:
pi_cc

tensor([[4.4567e-15, 2.0429e-05, 3.7457e-06],
        [0.0000e+00, 0.0000e+00, 1.2777e-02],
        [0.0000e+00, 0.0000e+00, 9.8720e-01]])

In [20]:
n_cc

tensor([[3.7190e-21, 1.7048e-11, 3.1256e-12],
        [0.0000e+00, 0.0000e+00, 1.0662e-08],
        [0.0000e+00, 0.0000e+00, 8.2378e-07]])

In [21]:
pi_cc

tensor([[4.4567e-15, 2.0429e-05, 3.7457e-06],
        [0.0000e+00, 0.0000e+00, 1.2777e-02],
        [0.0000e+00, 0.0000e+00, 9.8720e-01]])

In [22]:
n_c

tensor([37.0421, 50.0000, 62.9579])

In [ ]:
pi_c

In [15]:
#tensor([[6.8500, 3.0737, 5.7421, 2.0711],
#        [5.0060, 3.4280, 1.4620, 0.2460],
#        [5.9016, 2.7484, 4.3935, 1.4339]], dtype=torch.float64)
mu_c

tensor([[6.7986, 3.0524, 5.7335, 2.0876],
        [5.7689, 3.2681, 3.2798, 1.0297],
        [5.8433, 3.0573, 3.7580, 1.1993]], dtype=torch.float64)

In [16]:
#tensor([[[0.2442, 0.0254, 0.1773, 0.0055],
#         [0.0254, 0.0842, 0.0122, 0.0344],
#         [0.1773, 0.0122, 0.2387, 0.0253],
#         [0.0055, 0.0344, 0.0253, 0.0783]],

#        [[0.1242, 0.0992, 0.0164, 0.0103],
#         [0.0992, 0.1437, 0.0117, 0.0093],
#         [0.0164, 0.0117, 0.0302, 0.0061],
#         [0.0103, 0.0093, 0.0061, 0.0111]],

#        [[0.2175, 0.0637, 0.1343, 0.0345],
#         [0.0637, 0.0878, 0.0549, 0.0290],
#         [0.1343, 0.0549, 0.2590, 0.1214],
#         [0.0345, 0.0290, 0.1214, 0.0885]]])

co_c

tensor([[[0.3064, 0.0464, 0.2257, 0.0087],
         [0.0464, 0.0929, 0.0299, 0.0367],
         [0.2257, 0.0299, 0.2502, 0.0147],
         [0.0087, 0.0367, 0.0147, 0.0751]],

        [[0.2003, 0.0756, 0.1052, 0.0095],
         [0.0756, 0.1204, 0.0193, 0.0208],
         [0.1052, 0.0193, 0.1234, 0.0097],
         [0.0095, 0.0208, 0.0097, 0.0382]],

        [[0.2191, 0.0757, 0.1310, 0.0268],
         [0.0757, 0.1075, 0.0398, 0.0266],
         [0.1310, 0.0398, 0.1914, 0.0589],
         [0.0268, 0.0266, 0.0589, 0.0563]]])

In [237]:
## update mu_cj
top1 = 0; bot1 = 0
top2 = 0; bot2 = 0
for j in range(c):

  for i in range(n):
    
    w1 = r_i[i] * r_ij[j,i]
    bot1 += w1
    top1 += torch.tensor([X.iloc[i,:]]) * w1
    
    for k in range(c):
      w2 = (1-r_i[i]) * r_ijk[j,k,i]
      bot2 += w2
      top2 += (torch.tensor([X.iloc[i,:]]) - mu_c[k,:]) * w2
      
  mu_c[j] = (top1 + top2) / (bot1 + bot2)

## update sigma_cj
top1 = 0; bot1 = 0
top2 = 0; bot2 = 0
for j in range(c):
  
  for i in range(n):
    top_ker1 = torch.tensor([X.iloc[i,:]]) - mu_c[j,:]
    w1 = r_i[i] * r_ij[j,i]
    bot1 += w1
    top1 += w1 * (top_ker1.T * top_ker1)

    top_ker2 = torch.tensor([X.iloc[i,:]]) - (mu_c[j,:] + mu_c[k,:])    
    for k in range(c):
      w2 = (1-r_i[i]) * r_ijk[j,k,i]
      bot2 += w2
      top2 += w2 * ((top_ker2.T * top_ker2) - co_c[k,:])
  
  co_c[j] = (top1 + top2) / (bot1 + bot2)

In [209]:
top1 = 0; bot1 = 0
top2 = 0; bot2 = 0
for j in range(c):
  
  for i in range(n):
    top_ker1 = torch.tensor([X.iloc[i,:]]) - mu_c[j,:]
    w1 = r_i[i] * r_ij[j,i]
    bot1 += w1
    top1 += w1 * (top_ker1.T * top_ker1)

    top_ker2 = torch.tensor([X.iloc[i,:]]) - (mu_c[j,:] + mu_c[k,:])    
    for k in range(c):
      w2 = (1-r_i[i]) * r_ijk[j,k,i]
      bot2 += w2
      top2 += w2 * ((top_ker2.T * top_ker2) - co_c[k])
  co_c[j] = (top1 + top2) / (bot1 + bot2)

In [ ]:


## EM algorithm
llv = [0.0]
for iter in range(itermax):
  lli = estep()
  mstep()
      
  print('Iteration', iter + 1, 'Likelihood: ', lli)
        
  if abs(llv[-1] - lli) < 1e-4:
    break 
      
  llv.append(lli)
